In [1]:

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
from dqn import DQNAgent
from environment import environment, candle_class
from transformer_layer import TransformerBlock, PositionEmbedding

import time
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd    
import numpy as np
from collections import deque
from tqdm import tqdm
import random
import cv2
import pickle

name = "dqn_trading_transformer"
data_dir = "./archive"
resume = True
#resume = False

warmup_parallel = 8
train_parallel = 8
warmup_steps = 2000

lr = 0.0001
memory_size = 50000
gamma = 0.99
exploration = 0.02
target_model_sync = 100
batch_size = 24

dlen = 120
pos_size = 0.05 * 100000
comm = 15/100000
res_high = 100

x = environment(data_dir, dlen, res_high, comm, pos_size)
m5,m15,h1,h4,d1,pos = x.reset(True)

while True:
    obs, _,_ = x.step(0)
    m5,m15,h1,h4,d1,pos = obs
    m5 +=1 
    m5/=2
    m15 +=1 
    m15/=2
    h1 +=1 
    h1/=2
    h4 +=1 
    h4/=2
    d1 +=1 
    d1/=2
    cv2.imshow("m5",cv2.resize(m5,(300,300)))
    cv2.imshow("m15",cv2.resize(m15,(300,300)))
    cv2.imshow("h1",cv2.resize(h1,(300,300)))
    cv2.imshow("h4",cv2.resize(h4,(300,300)))
    cv2.imshow("d1",cv2.resize(d1,(300,300)))
    cv2.waitKey(10)

In [2]:


tf.keras.backend.clear_session()

def make_model():
    C1 = tf.keras.layers.Conv2D(64, 9,activation="relu", padding="same")

    D1 = tf.keras.layers.Dense(64)
    D2 = tf.keras.layers.Dense(512)
    D3 = tf.keras.layers.Dense(256)
    num_tx_units = 96
    D4 = tf.keras.layers.Dense(num_tx_units)
    D5 = tf.keras.layers.Dense(512)
    D6 = tf.keras.layers.Dense(256)

    LR = tf.keras.layers.LeakyReLU()

    LN1 = tf.keras.layers.LayerNormalization()

    P1 =  PositionEmbedding(dlen+1, num_tx_units)
    T1 = TransformerBlock(num_tx_units, 8, 256)
    T2= TransformerBlock(num_tx_units, 8, 256)
    T3 = TransformerBlock(num_tx_units, 8, 256)

    cn = tf.keras.layers.Concatenate()

    GR1 = tf.keras.layers.GRU(256)
    def proc_chart(x):
        #x1 = image
        #x2 = time
        x1 = x[::, :-1, :]
        x2 = x[::,-1,:]

        x1 = tf.keras.layers.Reshape((res_high, dlen+1, 1))(x1)

        x5 = C1(x1)
        x1 =cn([x1,x5])
        x1 = D1(x1)

        x1 = tf.transpose(x1,perm=[0, 2, 1, 3])
        x1 = tf.keras.layers.Reshape((dlen+1, res_high*x1.shape[-1]))(x1)
        x2 = tf.keras.layers.Reshape((dlen+1, 1))(x2)
        x1 = cn([x1,x2])

        x1 = D2(x1)
        x1 = LR(x1)
        x1 = D3(x1)
        x1 = LR(x1)
        x1 = D4(x1)
        x1 = LR(x1)
        x1 = LN1(x1)



        x1 = P1(x1)
        x1 = T1(x1)
        x1 = T2(x1)
        x1 = T3(x1)

        x1 = D5(x1)
        x1 = LR(x1)
        x1 = D6(x1)
        x1 = LR(x1)
        #x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x1 = GR1(x1)

        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(256,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        #x1 = tf.keras.layers.LayerNormalization()(x1)
        return x1
    
    input_m5 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_m15 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_h1 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_h4 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_d1 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    
    x0 = proc_chart(input_m5)
    x1 = proc_chart(input_m15)
    x2 = proc_chart(input_h1)
    x3 = proc_chart(input_h4)
    x4 = proc_chart(input_d1)
    
    input_net_position = tf.keras.layers.Input(shape = (1))


    x =cn([x0,x1,x2,x3,x4,input_net_position])
    
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    
    outputs = tf.keras.layers.Dense(3, activation = "linear", use_bias=False, dtype="float32")(x)
    model = tf.keras.Model([input_m5,input_m15,input_h1,input_h4, input_d1, input_net_position], outputs)
    return model
    
model = make_model()
target_model = make_model() #tf.keras.models.clone_model(model) does not work on shared layers
#model.summary()

In [3]:
opt = tf.keras.optimizers.Adam(lr)




agent = DQNAgent(
    model = model, 
    target_model = target_model,
    n_actions = 3, 
    memory_size = memory_size, 
    gamma=gamma,
    optimizer = opt,
    batch_size = batch_size, 
    target_model_sync = target_model_sync,
    exploration = exploration,
    name=name,
    output_dir = "tx_3/")

if resume:
	print("loading weights...")
	agent.load_weights()

loading weights...


In [4]:
x = [environment(data_dir, dlen, res_high, comm, pos_size, False) for _ in range(warmup_parallel)]
print("warmup...")
n = warmup_steps
agent.train(num_steps = n, envs = x, warmup = n, log_interval = n)

warmup...
using ./archive/EURGBP.csv_m5.csv_candle_classes
using ./archive/USDJPY.csv_m5.csv_candle_classes
using ./archive/USDCHF.csv_m5.csv_candle_classes
using ./archive/EURAUD.csv_m5.csv_candle_classes
using ./archive/GBPJPY.csv_m5.csv_candle_classes
using ./archive/USDCAD.csv_m5.csv_candle_classes
using ./archive/AUDUSD.csv_m5.csv_candle_classes
using ./archive/EURJPY.csv_m5.csv_candle_classes
2000/2000 [==============================] - 252s 123ms/step - loss: 0.0000e+00 - mean q: 0.0000e+00 - rewards: -0.0251 - t: 119.8273


c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
len(agent.memory)

16000

In [ ]:
x = [environment(data_dir, dlen, res_high, comm, pos_size, False) for _ in range(train_parallel)]
print("training...")
n = 1000000000
agent.train(num_steps = n, envs = x, warmup = 0, log_interval = 1000)
print("done")

training...
using ./archive/GBPJPY.csv_m5.csv_candle_classes
using ./archive/EURAUD.csv_m5.csv_candle_classes
using ./archive/GBPUSD.csv_m5.csv_candle_classes
using ./archive/AUDJPY.csv_m5.csv_candle_classes
using ./archive/GBPCHF.csv_m5.csv_candle_classes
using ./archive/USDCAD.csv_m5.csv_candle_classes
using ./archive/EURAUD.csv_m5.csv_candle_classes
using ./archive/CHFJPY.csv_m5.csv_candle_classes
 888/1000 [=========================>....] - ETA: 1:22 - loss: 3.6169 - mean q: 10.1559 - rewards: -0.0060 - t: 755.1474

In [ ]:
rew = [i[2] for i in agent.memory]
sorted(rew)[0:10], sorted(rew)[-10:][::-1], " - ", np.mean([abs(i) for i in rew])

In [ ]:
agent.save_weights()